# Dengue Analysis:
---

In [ ]:
# Standard libraries
import os
import sys
import warnings
import zipfile
import subprocess

# Third-party libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import shap
import optuna
import lightgbm as lgb
from lightgbm import LGBMClassifier

# Scikit-learn - Preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Scikit-learn - Models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample

# Scikit-learn - Validation and Metrics
from sklearn.model_selection import (
    train_test_split,
    StratifiedKFold,
    cross_val_score,
    cross_validate
)
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score
)

# Custom utilities
from utils.utils import (
    print_with_colors,
    is_int,
    process_num_like_cols,
    print_with_multiple_columns
)

## Download:
---
* Download the Dataset from Web if not already downloaded.

In [ ]:
if not os.path.exists("./raw_data"):
    os.makedirs("./raw_data")

if not os.path.exists("./raw_data/arbovirus_clinical_data"):
    # Download of .zip file
    url = "https://prod-dcd-datasets-cache-zipfiles.s3.eu-west-1.amazonaws.com/2d3kr8zynf-4.zip"
    output = f"./raw_data/dataset.zip"
    subprocess.run(["wget", "--quiet", "--no-check-certificate", url, "-O", output])

    # Extraction of .zip file
    subprocess.run(["unzip", output])
    subprocess.run(["mv", "./2d3kr8zynf-4", "./raw_data/arbovirus_clinical_data"])
    subprocess.run(["rm", output])

## Reading the Dataset:
---
* Using `chunksize` on `pd.read_csv()` method to use less RAM memory during reading

<font color='yellow'>Note: if you already have the .parquet file, you can skip to [this](#file) section<font>

In [ ]:
import warnings

missing_values = [
    '', ' ', 'NA', 'N/A', 'NULL',
    'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT',
    'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI',
    'DT_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA',
    'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS',
    'DT_INVEST', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA',
    'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE',
    'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO',
    'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA',
    'ACIDO_PEPT', 'AUTO_IMUNE', 'RESUL_SORO', 'RESUL_NS1', 'RESUL_VI_N',
    'RESUL_PCR_', 'HISTOPA_N', 'IMUNOH_N', 'HOSPITALIZ', 'TPAUTOCTO',
    'COUFINF', 'COPAISINF', 'COMUNINF', 'CLASSI_FIN', 'EVOLUCAO', 'DT_ENCERRA', '.'
]

warnings.filterwarnings("ignore")
# Low memory safe reading of the CSV file
splitted_df = pd.read_csv(
    './raw_data/arbovirus_clinical_data/dengue.csv',
    sep=',',
    header=0,
    na_values=missing_values,
    chunksize=100_000,
)

# Concatenate all chunks into a single DataFrame
dengue_df = pd.concat(splitted_df, ignore_index=True)
warnings.filterwarnings("default")

* The file `attributes.csv` has important information about the features

In [ ]:
attributes = pd.read_csv("raw_data/arbovirus_clinical_data/attributes.csv", sep=",", header=0, low_memory=False)
attributes = attributes.ffill()
attributes = attributes.groupby(["Attribute", "Description"])["Value"].apply('; '.join).reset_index(name="Values")

## Pre Processing
---
### Null Data Removal:
* Features with frequency > 60% of null values are dropped.
* Also, columns like `["CS_FLXRET", "TP_SISTEMA", "CRITERIO", "TP_NOT", "Unnamed: 0"]` doesn't have useful information, therefore they can be dropped.

In [ ]:
dengue_df = dengue_df.loc[:, dengue_df.isnull().mean() < .60]
dengue_df = dengue_df.drop(columns=["CS_FLXRET", "TP_SISTEMA", "CRITERIO", "TP_NOT", "Unnamed: 0"])

* Printing unique values for each feature to check their data type.

In [ ]:
for col in dengue_df.columns.to_list():
    if str(col) in attributes["Attribute"].to_list():
        print(f"Column '{col}' has {dengue_df[col].unique().size} unique values.")
        if dengue_df[col].unique().size < 50:
            print(dengue_df[col].unique(), end="\n\n")
        else:
            print("To many unique values, skipping...", end="\n\n")
    else:
        print_with_colors(f"Column '{col}' not in attributes. Skipping display...", "yellow", end="\n\n")

* Correlation matrix to check for dependent or exclusive variables.

In [ ]:
numeric_df = dengue_df.select_dtypes(include=['number'])
correlation_matrix = numeric_df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Plotar o heatmap
plt.figure(figsize=(24, 20))
sns.heatmap(correlation_matrix, annot=True, mask=mask, fmt=".2f", cmap="Blues", square=True)
plt.title("Matriz de Correlação - Variáveis Numéricas")
plt.tight_layout()
plt.show()

* Count of `NU_ANO` using percents

In [ ]:
# percentual counting
percents = dengue_df['NU_ANO'].value_counts(normalize=True) * 100

# Format to 2 decimal places
percents = percents.round(2)

print(percents)

* Distribution of cases per year and `CLASSI_FIN`

In [ ]:
plt.figure(figsize=(25, 6))
sns.countplot(data=dengue_df, x='NU_ANO')
plt.title('Distribution of cases per year and final classification')
plt.xlabel('Year')
plt.ylabel('Number of cases')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

* Distribution of cases per `CLASSI_FIN`

In [ ]:
dengue_df['CLASSI_FIN'] = dengue_df[dengue_df['CLASSI_FIN']]

In [ ]:
dengue_df = dengue_df[dengue_df['CLASSI_FIN'] != 6.]

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=dengue_df, x='CLASSI_FIN')
plt.title('Distribution of cases per final classification')
plt.xlabel('Final Classification')
plt.ylabel('Number os cases')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

### Standardization of column values:
* Since the system has changed over the year, multiple codes were used to represent some types of Dengue. In the cell below we standardized these problem.

In [ ]:
dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].astype('object')

# Mappings to 'Dengue'
dengue_df.loc[dengue_df['CLASSI_FIN'].isin([1, 10]), 'CLASSI_FIN'] = 'Dengue'

# Mappings to other classes
dengue_df.loc[dengue_df['CLASSI_FIN'].isin([3, 4, 12]), 'CLASSI_FIN'] = 'Dengue Grave'
dengue_df.loc[dengue_df['CLASSI_FIN'].isin([2, 11]), 'CLASSI_FIN'] = 'Dengue com sinais de alarme'
dengue_df.loc[dengue_df['CLASSI_FIN'].isin([5, 6, 8]), 'CLASSI_FIN'] = 'Discarded/Inconclusive'

# Filling nan values with the negative class
dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].fillna('Discarded/Inconclusive')

# Converting to category dtype
dengue_df = dengue_df[dengue_df['CLASSI_FIN'].isin(['Dengue', 'Discarded/Inconclusive'])]
dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].astype('category')

dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].cat.remove_unused_categories()

In [ ]:
# convert age code to years
def extract_age_in_years(x):
    tipo = x // 1000
    valor = x % 1000
    return valor if tipo == 4 else valor / 12 if tipo == 3 else 0

dengue_df['age_years'] = dengue_df['NU_IDADE_N'].apply(extract_age_in_years).astype(int)
dengue_df = dengue_df[dengue_df["age_years"] <= 100]

* Plotting the distribution of cases per final classification

In [ ]:
plt.figure(figsize=(25, 6))
sns.countplot(data=dengue_df, x='age_years', hue='CLASSI_FIN', palette='Set2')
plt.title('Distribution of cases per final classification')
plt.xlabel('Age')
plt.ylabel('Number of cases')
plt.legend(title='Final Classification')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
def classify_age_group(age_years):
    if pd.isnull(age_years):
        return 'Ignored'
    elif age_years < 1:
        return '<1 year'
    elif 1 <= age_years <= 4:
        return '1-4 years'
    elif 5 <= age_years <= 9:
        return '5-9 years'
    elif 10 <= age_years <= 14:
        return '10-14 years'
    elif 15 <= age_years <= 19:
        return '15-19 years'
    elif 20 <= age_years <= 39:
        return '20-39 years'
    elif 40 <= age_years <= 59:
        return '40-59 years'
    elif age_years >= 60:
        return '60+ years'
    else:
        return 'Ignored'

In [ ]:
dengue_df['age_range'] = dengue_df['age_years'].apply(classify_age_group)

In [ ]:
dengue_df['age_range'].unique()

In [ ]:
plt.figure(figsize=(15, 6))
sns.countplot(data=dengue_df, x='age_range', hue='CLASSI_FIN', palette='Set2')
plt.title('Distribution of final classifications per age group')
plt.xlabel('Age Group')
plt.ylabel('Number of cases')
plt.legend(title='Final Classification')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
age_order = [
    '<1 year',
    '1-4 years',
    '5-9 years',
    '10-14 years',
    '15-19 years',
    '20-39 years',
    '40-59 years',
    '60+ years'
]

plt.figure(figsize=(15, 6))
sns.countplot(data=dengue_df, x='age_range', hue='CLASSI_FIN', palette='Set2', order=age_order)
plt.title('Distribution of final classifications per age group')
plt.xlabel('Age Group')
plt.ylabel('Number of cases')
plt.legend(title='Final Classification')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Replace NaN by a label
dengue_df['CS_SEXO_PLOT'] = dengue_df['CS_SEXO'].fillna('Ignored')

plt.figure(figsize=(10, 6))
sns.countplot(data=dengue_df, x='CS_SEXO_PLOT', palette='Set2')
plt.title('Distribution of cases per sex')
plt.xlabel('Sex')
plt.ylabel('Number of cases')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
mapping_uf_to_region = {
    11: 'Norte', 12: 'Norte', 13: 'Norte', 14: 'Norte', 15: 'Norte', 16: 'Norte', 17: 'Norte',
    21: 'Nordeste', 22: 'Nordeste', 23: 'Nordeste', 24: 'Nordeste', 25: 'Nordeste',
    26: 'Nordeste', 27: 'Nordeste', 28: 'Nordeste', 29: 'Nordeste',
    31: 'Sudeste', 32: 'Sudeste', 33: 'Sudeste', 35: 'Sudeste',
    41: 'Sul', 42: 'Sul', 43: 'Sul',
    50: 'Centro-Oeste', 51: 'Centro-Oeste', 52: 'Centro-Oeste', 53: 'Centro-Oeste'
}

# Aplica o mapeamento
dengue_df['REGION'] = dengue_df['SG_UF'].map(mapping_uf_to_region)

In [ ]:
dengue_df = dengue_df.dropna(subset=['REGION'])

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=dengue_df, x='REGION', hue='CLASSI_FIN', palette='Set2')
plt.title('Distribution of cases per region and final classification')
plt.xlabel('REGION')
plt.ylabel('Númber of cases')
plt.legend(title='Final classification')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

Removing useless columns, such as `ID_AGRAVO` which has only 1 fixed value: `A90`;

In [ ]:
dengue_df = dengue_df.drop(columns=["CS_ESCOL_N", "NU_IDADE_N", "DT_NASC", "DT_SIN_PRI", "DT_NOTIFIC", "DT_INVEST", "DT_ENCERRA"])
dengue_df = dengue_df.drop(columns=["Unnamed: 0", "Unnamed: 0.1"], errors='ignore')
dengue_df = dengue_df.drop(columns=['ID_AGRAVO'])
dengue_df = dengue_df.drop(columns=['SG_UF', 'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS'])
dengue_df = dengue_df.drop(columns=['CS_RACA', 'COMUNINF', 'COPAISINF', 'COUFINF', 'COUFINF'])
dengue_df = dengue_df.drop(columns=['SEM_NOT', 'NU_ANO', 'RESUL_SORO', 'RESUL_NS1', 'RESUL_VI_N', 'RESUL_PCR_', 'HISTOPA_N', 'IMUNOH_N','EVOLUCAO', 'TPAUTOCTO', 'HOSPITALIZ', 'idade_anos'])
dengue_df = dengue_df.drop(columns=['SEM_PRI'])
dengue_df.info()

### Null data padding with default values:
The resulting attributes that still had null data were entered with the default values referring to the data dictionary.

In [ ]:
dtypes = {
    'SEM_NOT': 'category',
    'NU_ANO': 'int16',
    'SG_UF_NOT': 'category',
    'ID_MUNICIP': 'category',
    'ID_REGIONA': 'category',
    'ID_UNIDADE': 'category',
    'SEM_PRI': 'int32',
    'NU_IDADE_N': 'int8',
    'CS_SEXO': 'category',
    'CS_GESTANT': 'category',
    'CS_RACA': 'category',
    'CS_ESCOL_N': 'category',
    'SG_UF': 'category',
    'ID_MN_RESI': 'category',
    'ID_RG_RESI': 'category',
    'ID_PAIS': 'category',
    'FEBRE': 'category',
    'MIALGIA': 'category',
    'CEFALEIA': 'category',
    'EXANTEMA': 'category',
    'VOMITO': 'category',
    'NAUSEA': 'category',
    'DOR_COSTAS': 'category',
    'CONJUNTVIT': 'category',
    'ARTRITE': 'category',
    'ARTRALGIA': 'category',
    'PETEQUIA_N': 'category',
    'LEUCOPENIA': 'category',
    'LACO': 'category',
    'DOR_RETRO': 'category',
    'DIABETES': 'category',
    'HEMATOLOG': 'category',
    'HEPATOPAT': 'category',
    'RENAL': 'category',
    'HIPERTENSA': 'category',
    'ACIDO_PEPT': 'category',
    'AUTO_IMUNE': 'category',
    'RESUL_SORO': 'category',
    'RESUL_NS1': 'category',
    'RESUL_VI_N': 'category',
    'RESUL_PCR_': 'category',
    'HISTOPA_N': 'category',
    'IMUNOH_N': 'category',
    'HOSPITALIZ': 'category',
    'TPAUTOCTO': 'category',
    'COUFINF': 'category',
    'COPAISINF': 'category',
    'COMUNINF': 'category',
    'EVOLUCAO': 'category',
}

In [ ]:
dengue_df = process_num_like_cols(dengue_df)

### Setting dtypes to columns:

In [ ]:
# date_cols = ['DT_NOTIFIC', 'DT_SIN_PRI', 'DT_NASC', 'DT_INVEST', 'DT_ENCERRA']
# for col in date_cols:
#     if col in dengue_df.columns:
#         dengue_df[col] = pd.to_datetime(dengue_df[col], errors='coerce')

# dengue_df['SEM_PRI'] = dengue_df['SEM_PRI'].apply(lambda x: x.replace('-', '') if isinstance(x, str) else x)

# exam_cols = [
#     "RESUL_SORO",
#     "RESUL_NS1",
#     "RESUL_VI_N",
#     "RESUL_PCR_",
#     "HISTOPA_N",
#     "IMUNOH_N"
# ]
# for col in exam_cols:
#     if dengue_df[col].isnull().sum() > 0:
#         dengue_df.loc[dengue_df[col].isnull(), col] = 4

# dengue_df['CS_SEXO'] = dengue_df['CS_SEXO'].fillna('I')

# # In the other attributes, the value of "not informed" is 9.
# columns_to_be_filled = [
#     col
#     for col in dengue_df.columns
#     if col not in exam_cols
#     and 'DT_' not in str(col) # for datetime columns it doesn't make sense
#     and not 'CS_SEXO'.__eq__(str(col)) # CS_SEXO has the special value 'I' for NaNs
# ]
# for col in columns_to_be_filled:
#     if dengue_df[col].isnull().sum() > 0:
#         dengue_df.loc[dengue_df[col].isnull(), col] = 9

# dengue_df = dengue_df.astype(dtypes)

dengue_df['CS_SEXO'] = dengue_df['CS_SEXO'].replace('I', np.nan)
dengue_df = dengue_df.dropna(subset=['CS_SEXO'])
dengue_df.info(show_counts=True)

### Data normalization and evaluation

In [ ]:
# Binarize CS_GESTANT
def binarize_gestant(x):
    if pd.isnull(x) or x == 9:
        return 0
    elif x in [1, 2, 3, 4]:
        return 1
    else:
        return 0

dengue_df['CS_GESTANT'] = dengue_df['CS_GESTANT'].apply(binarize_gestant)

# Consider age_group and region as categories
dengue_df['age_range'] = dengue_df['age_range'].astype('category')
dengue_df['REGION'] = dengue_df['REGION'].astype('category')

# Generate dummies with dummy_na=True (keep NaNs as explicit category)
dengue_df = pd.get_dummies(dengue_df, columns=['age_range', 'REGION'], dummy_na=True)

In [ ]:
dengue_df['CS_GESTANT'].value_counts()

In [ ]:
# Binarize target class
dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].map({'Dengue': 1, 'Discarded/Inconclusive': 0})
dengue_df['CLASSI_FIN'] = dengue_df['CLASSI_FIN'].astype('int')

In [ ]:
numeric_df = dengue_df.select_dtypes(include=['number', 'bool'])
correlation_matrix = numeric_df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Plot the heatmap
plt.figure(figsize=(24, 20))
sns.heatmap(correlation_matrix, annot=True, mask=mask, fmt=".2f", cmap="Blues", square=True)
plt.title("Correlation Matrix - Numeric Features")
plt.tight_layout()
plt.show()

In [ ]:
dengue_df.drop(columns=['REGION_nan', 'REGION_Sul', 'age_range_nan', 'age_range_<1 year'], inplace=True)

### Saving processed DataFrame to parquet:

In [ ]:
dengue_df.to_parquet("./preprocessed_data/dengue.parquet")

## Reading Parquet File:
---
* Uncomment the cell below if you already have the `.parquet` file and just want to load it.

In [ ]:
dengue_df = pd.read_parquet("./preprocessed_data/dengue.parquet")

dtypes = {
    'SEM_NOT': 'int32', 'NU_ANO': 'int16', 'SG_UF_NOT': 'category', 'ID_MUNICIP': 'category',
    'ID_REGIONA': 'category', 'ID_UNIDADE': 'category', 'SEM_PRI': 'int32',
    'NU_IDADE_N': 'int8', 'CS_SEXO': 'category', 'CS_GESTANT': 'category',
    'CS_RACA': 'category', 'CS_ESCOL_N': 'category', 'SG_UF': 'category',
    'ID_MN_RESI': 'category', 'ID_RG_RESI': 'category', 'ID_PAIS': 'category',
    'FEBRE': 'category', 'MIALGIA': 'category', 'CEFALEIA': 'category',
    'EXANTEMA': 'category', 'VOMITO': 'category', 'NAUSEA': 'category',
    'DOR_COSTAS': 'category', 'CONJUNTVIT': 'category', 'ARTRITE': 'category',
    'ARTRALGIA': 'category', 'PETEQUIA_N': 'category', 'LEUCOPENIA': 'category',
    'LACO': 'category', 'DOR_RETRO': 'category', 'DIABETES': 'category',
    'HEMATOLOG': 'category', 'HEPATOPAT': 'category', 'RENAL': 'category',
    'HIPERTENSA': 'category', 'ACIDO_PEPT': 'category', 'AUTO_IMUNE': 'category',
    'RESUL_SORO': 'category', 'RESUL_NS1': 'category', 'RESUL_VI_N': 'category',
    'RESUL_PCR_': 'category', 'HISTOPA_N': 'category', 'IMUNOH_N': 'category',
    'HOSPITALIZ': 'category', 'TPAUTOCTO': 'category', 'COUFINF': 'category',
    'COPAISINF': 'category', 'COMUNINF': 'category', 'EVOLUCAO': 'category',
}

dengue_df = dengue_df.astype(dtypes)

## Removing Data Leakage Features:
---
Some features in the dataset have information from the future (*i.e.* after the `CLASSI_FIN` has been diagnosed)

In [ ]:
leaky_columns = [
    'RESUL_SORO', 'RESUL_NS1', 'RESUL_VI_N', 'RESUL_PCR_', 'HISTOPA_N',
    'IMUNOH_N', 'EVOLUCAO', 'DT_ENCERRA', 'TPAUTOCTO', 'COUFINF', 'COPAISINF',
    'COMUNINF', 'CODISINF', 'CO_BAINFC', 'NOBAIINF', 'DOENCA_TRA', 'DT_OBITO',
    
]
leaky_columns = [col for col in leaky_columns if col in dengue_df.columns]

dengue_df = dengue_df.drop(columns=leaky_columns)

# Already removed
# dengue_df = dengue_df[~dengue_df["NU_IDADE_N"] < 0] # Age < 0 don't make sense

## Feature Engineering
---
* Trying to extract useful information from features:

In [ ]:
cols = [
    'CS_SEXO','CS_GESTANT', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 
    'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 
    'DOR_RETRO', 'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT', 
    'AUTO_IMUNE'
]

dengue_df['CS_SEXO'] = dengue_df['CS_SEXO'].map({'M': 1, 'F': 0})

# Treatment as discussed:
# - For CS_SEXO: create category 2 for nulls
# dengue_df['CS_SEXO'] = dengue_df['CS_SEXO'].fillna(2).astype(int)

# - For CS_GESTANT: create category 0 for nulls
dengue_df['CS_GESTANT'] = dengue_df['CS_GESTANT'].fillna(0).astype(int)

# - For symptoms and comorbidities: binarize the columns (1 for yes, 0 for no/other)
symptoms_comorbidities = cols.copy()
symptoms_comorbidities.remove('CS_SEXO')
symptoms_comorbidities.remove('CS_GESTANT')

for col in symptoms_comorbidities:
    # Set to 1 if the original value is 1, otherwise set to 0
    dengue_df[col] = dengue_df[col].apply(lambda x: 1 if x == 1 else 0)

In [ ]:
dengue_df['FEBRE'].value_counts(dropna=False)

In [ ]:
# Columns with error: 'DT_SIN_PRI' cannot be equal to 'DT_NASC'.
dengue_df = dengue_df[~(dengue_df['DT_NASC'] == dengue_df['DT_SIN_PRI'])]

# Time gap between notification and first simptoms.
dengue_df['time_until_report'] = (dengue_df['DT_NOTIFIC'] - dengue_df['DT_SIN_PRI']).dt.days

# Negative values for 'time_until_report' or 'time_until_report' >= 30 doesn't make sense, since the virus
# expresses their simptoms in a shorter period of time.
dengue_df = dengue_df.loc[(dengue_df['time_until_report'] <= 30) & (dengue_df['time_until_report'] >= 0)]

# 'SEM_NOT' is the epidemiological week of the notification date.
# We don't need the year, so we can convert it to a string and remove the first characters.
dengue_df['SEM_NOT'] = dengue_df['SEM_NOT'].astype('str')
dengue_df['SEM_NOT'] = dengue_df['SEM_NOT'].apply(lambda x: x[4:] if len(x) > 4 else x[2:])
dengue_df['SEM_NOT'] = dengue_df['SEM_NOT'].astype('category')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.boxplot(x=dengue_df['time_until_report'], orient='horizontal')
plt.title('Distribuição de "time_until_report"')
plt.xlabel('Dias entre Sintomas e Notificação')
plt.show()

In [ ]:
numeric_df = dengue_df.select_dtypes(include=['number', 'bool'])
correlation_matrix = numeric_df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Plotar o heatmap
plt.figure(figsize=(24, 20))
sns.heatmap(correlation_matrix, annot=True, mask=mask, fmt=".2f", cmap="Blues", square=True)
plt.title("Correlation Matrix - Numeric Features")
plt.tight_layout()
plt.show()

In [ ]:
dengue_df = dengue_df.drop(columns=["CS_SEXO_PLOT"], errors='ignore')

In [ ]:
attributes = pd.read_csv("raw_data/arbovirus_clinical_data/attributes.csv", sep=",", header=0, low_memory=False)
attributes = attributes.ffill()
attributes = attributes.groupby(["Attribute", "Description"])["Value"].apply('; '.join).reset_index(name="Values")
attributes = attributes[~attributes["Attribute"].isin(leaky_columns)]
attributes = attributes[attributes["Attribute"].isin(dengue_df.columns)]
attributes = attributes.reset_index(drop=True)

acido_pept = {
    "Attribute": "ACIDO_PEPT",
    "Description": "Pre-existing disease - Acid peptic disease",
    "Values": "1: Yes; 2: No",
}

time_until_report = {
    "Attribute": "time_until_report",
    "Description": "Time until report",
    "Values": "0: 0 days; 1: 1 day; 2: 2 days; ...; 30: 30 days",
}
attributes = pd.concat([attributes, pd.DataFrame([acido_pept]), pd.DataFrame([time_until_report])], ignore_index=True)

attributes.to_csv("preprocessed_data/attributes.csv", index=False)

## First attempt to train and test (using only LightGBM):
---

In [ ]:
# Label encoding the target variable
le = LabelEncoder()
y = le.fit_transform(dengue_df['CLASSI_FIN'])

In [ ]:
# Keep mapping of target variable
target_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(f"Target mapping: {target_mapping}")

In [ ]:
# Preparing the feature matrix
X = dengue_df.drop(columns=['CLASSI_FIN'])

In [ ]:
# Extracting month and day of the week from the notification date
if 'DT_NOTIFIC' in X.columns:
    X['notif_month'] = X['DT_NOTIFIC'].dt.month
    X['notif_week_day'] = X['DT_NOTIFIC'].dt.dayofweek

In [ ]:
# Remove raw date columns
cols_to_drop = [col for col in X.columns if 'DT_' in col]
X = X.drop(columns=cols_to_drop)

In [ ]:
print("Features to train with:")
print_with_multiple_columns(X.columns.tolist(), 5)

In [ ]:
# Double-checking for 'object' columns and converting them to appropriate types
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category')

### Hyperparameter Tuning In a Small Subset:
---
* Instead of using 12 million registries, we will do Hyperparameter Tuning using only 500k samples, to save time and computer power.

In [ ]:
# Setting optuna objective function
def objective_parallel(trial, X_inner, y_inner):
    """
    Versão otimizada para paralelismo. Cada trial usará apenas 1 core.
    """
    X_train, X_val, y_train, y_val = train_test_split(
        X_inner, y_inner, test_size=0.25, random_state=42, stratify=y_inner
    )
    
    params = {
        'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': len(np.unique(y_inner)),
        'verbosity': -1, 'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'n_jobs': 1,
    }

    model = lgb.LGBMClassifier(**params, n_estimators=1000)

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(15, verbose=False)]
    )

    preds = model.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    
    return f1

In [ ]:
# Testing on a small subset
# Stratified sampling for training...
dengue_df['strata'] = (
    dengue_df['NU_ANO'].astype(str) + '_' +
    dengue_df['CLASSI_FIN'].astype(str)
)

# We will use a large ~ 5% (500k) sample size to ensure we have enough data for training.
sample_size = 500_000
sample_ratio = sample_size / len(dengue_df)

_, df_subset = train_test_split(
    dengue_df,
    test_size=sample_ratio,
    stratify=dengue_df['strata'],
    random_state=42
)

print("Sample subset created with size:", len(df_subset))

df_subset = df_subset.drop(columns=['strata'])

In [ ]:
# Label encoding the target variable
le = LabelEncoder()
y = le.fit_transform(df_subset['CLASSI_FIN'])

# Keep mapping of target variable
target_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(f"Target mapping: {target_mapping}")

# Preparing the feature matrix
X = df_subset.drop(columns=['CLASSI_FIN'])

# Extracting month and day of the week from the notification date
if 'DT_NOTIFIC' in X.columns:
    X['notif_month'] = X['DT_NOTIFIC'].dt.month
    X['notif_week_day'] = X['DT_NOTIFIC'].dt.dayofweek
    
# Remove raw date columns
cols_to_drop = [col for col in X.columns if 'DT_' in col]
X = X.drop(columns=cols_to_drop)

# Double-checking for 'object' columns and converting them to appropriate types
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category')

In [ ]:
# Nested Cross Validation:
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=100)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=100)

outer_scores = []
best_params_per_fold = []

In [ ]:
print("Starting Nested Cross Validation...")

for i, (train_idx, test_idx) in enumerate(outer_cv.split(X, y)):
    print(f"\n--- Processing over external fold {i+1}/5 ---")
    
    X_train_outer, X_test_outer = X.iloc[train_idx], X.iloc[test_idx]
    y_train_outer, y_test_outer = y[train_idx], y[test_idx]

    study = optuna.create_study(direction='maximize')
    study.optimize(
        lambda trial: objective_parallel(trial, X_train_outer, y_train_outer),
        n_trials=20,  # You can adjust the number of trials based on your computational resources
        n_jobs=-1     # <-- Here we use all available cores for parallel execution
    )

    best_params = study.best_trial.params
    best_params['n_jobs'] = 1 
    print(f"Best parameters found: {best_params}")

    final_model = lgb.LGBMClassifier(**best_params, n_estimators=1000, random_state=42)
    
    final_model.fit(
        X_train_outer, y_train_outer,
        eval_set=[(X_test_outer, y_test_outer)],
        eval_metric='multi_logloss',
        callbacks=[lgb.early_stopping(15, verbose=False)]
    )
    
    y_pred_outer = final_model.predict(X_test_outer)
    score = f1_score(y_test_outer, y_pred_outer, average='macro')
    outer_scores.append(score)
    print(f"F1-Macro for External Fold: {i+1}: {score:.4f}")

In [ ]:
print("\n--- Final Evaluation of the Optimized Nested Cross-Validation ---")
print(f"Macro F1-Scores for each outer fold: {np.round(outer_scores, 4)}")
print(f"Mean Macro F1-Score: {np.mean(outer_scores):.4f}")
print(f"Macro F1-Score Standard Deviation: {np.std(outer_scores):.4f}")

In [ ]:
# --- SIMPLIFY WITH A SINGLE SPLIT ---
# We don't need Nested CV to find the leak
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# --- STEP 1: TRAIN A MODEL AND CHECK FEATURE IMPORTANCE ---
print("Training a LightGBM model for importance analysis...")
simple_lgbm = lgb.LGBMClassifier(objective='multiclass', random_state=42)
simple_lgbm.fit(X_train, y_train)

print("\n--- FEATURE IMPORTANCE PLOT (LGBM) ---")
print("Look for a bar that is MUCH taller than all the others.")
lgb.plot_importance(simple_lgbm, max_num_features=20, figsize=(10, 8),
                    importance_type='gain', title='Feature Importance (Gain)')
plt.show()

# The feature name at the top of the plot is our SUSPECT #1.

# --- STEP 2: VISUALIZE A SIMPLE DECISION TREE ---
print("\n--- SIMPLE DECISION TREE PLOT ---")
print("The feature at the top of the tree (root node) is the most likely culprit.")
# We need an X with only numeric columns for plot_tree
X_train_numeric = X_train.apply(pd.to_numeric, errors='coerce').fillna(0)

simple_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
simple_tree.fit(X_train_numeric, y_train)

plt.figure(figsize=(20, 12))
plot_tree(simple_tree,
          feature_names=X_train_numeric.columns,
          class_names=le.classes_,
          filled=True,
          rounded=True,
          fontsize=10)
plt.title("Simple Decision Tree to Identify the Dominant Feature")
plt.show()

## Second attempt to train and test (multiple models) - Final version (used in the article):
---

In [ ]:
# Select 10% of the data
dengue_sample = resample(dengue_df, replace=False, n_samples=int(len(dengue_df) * 0.1), random_state=42)

# Separate X and y
X = dengue_sample.drop(columns=['CLASSI_FIN', 'REGION_Sudeste', 'age_group_60+ years', 'age_group_40-59 years', 'age_group_20-39 years', 'age_group_15-19 years', 'age_group_10-14 years', 'age_group_5-9 years', 'age_group_1-4 years', 'REGION_Norte', 'REGION_Nordeste', 'REGION_Centro-Oeste'])
y = dengue_sample['CLASSI_FIN'].astype(int)

# Standardization for linear models
X_scaled = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)

# Cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Metrics
scoring = ['accuracy', 'f1_macro', 'precision_macro', 'recall_macro', 'roc_auc']

#### Random Forest

In [ ]:

model_rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
results_rf = cross_validate(model_rf, X, y, cv=cv, scoring=scoring, n_jobs=-1)

for metric in scoring:
    mean = results_rf[f'test_{metric}'].mean()
    std = results_rf[f'test_{metric}'].std()
    print(f"Random Forest {metric}: {mean:.4f} ± {std:.4f}")

In [ ]:
# Treinar Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X, y)

In [ ]:
# Importância das features
importances = rf.feature_importances_
feature_names = X.columns

# Organizar em DataFrame para visualização
feat_imp = pd.DataFrame({
    "Feature": feature_names,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

# Plot
plt.figure(figsize=(10, 8))
plt.barh(feat_imp["Feature"], feat_imp["Importance"])
plt.gca().invert_yaxis()
plt.title("Feature Importance - Random Forest")
plt.xlabel("Information Gain (Gini)")
plt.tight_layout()
plt.show()

#### Logistic Regression

In [ ]:


model_lr = LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
results_lr = cross_validate(model_lr, X_scaled, y, cv=cv, scoring=scoring, n_jobs=-1)

for metric in scoring:
    mean = results_lr[f'test_{metric}'].mean()
    std = results_lr[f'test_{metric}'].std()
    print(f"Logistic Regression {metric}: {mean:.4f} ± {std:.4f}")

#### SVC

In [ ]:


model_svm = LinearSVC(max_iter=1000, random_state=42)
results_svm = cross_validate(model_svm, X_scaled, y, cv=cv, scoring=scoring)

for metric in scoring:
    mean = results_svm[f'test_{metric}'].mean()
    std = results_svm[f'test_{metric}'].std()
    print(f"Linear SVM {metric}: {mean:.4f} ± {std:.4f}")

#### LGBM

In [ ]:


model_lgbm = LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1)
results_lgbm = cross_validate(model_lgbm, X, y, cv=cv, scoring=scoring, n_jobs=-1)

for metric in scoring:
    mean = results_lgbm[f'test_{metric}'].mean()
    std = results_lgbm[f'test_{metric}'].std()
    print(f"LightGBM {metric}: {mean:.4f} ± {std:.4f}")

#### Model extra evaluation

In [ ]:
X = dengue_sample.drop(columns=['CLASSI_FIN', 'REGION_Sudeste', 'age_group_60+ years', 'age_group_40-59 years', 'age_group_20-39 years', 'age_group_15-19 years', 'age_group_10-14 years', 'age_group_5-9 years', 'age_group_1-4 years', 'REGION_Norte', 'REGION_Nordeste', 'REGION_Centro-Oeste'])

In [ ]:
model = lgb.LGBMClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

##### SHAP

In [ ]:
explainer = shap.Explainer(model)
shap_values = explainer(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
shap.dependence_plot("CLASSI_", shap_values.values, X_test)

In [ ]:
shap.plots.waterfall(shap_values[0])